In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_csv('swat.csv')
#df.shape

#df = df.iloc[:5*len(df.index)//8]
#df = df.iloc[:len(df.index)//2]
df = df.iloc[:7500]
y = df["Normal/Attack"]
del df["Normal/Attack"]
df.tail(5)

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
7495,1451304295000000000,2.583352,510.4036,2,2,1,263.4261,8.370802,329.5309,2.442828,...,0.306505,2,1,249.5514,1.778105,188.5734,1.737353,1,2,1
7496,1451304296000000000,2.589117,510.2073,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306633,2,1,249.5514,1.778105,188.6535,1.737161,1,2,1
7497,1451304297000000000,2.592000,510.0896,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306953,2,1,249.6635,1.778105,188.6535,1.736584,1,2,1
7498,1451304298000000000,2.592000,510.1288,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306953,2,1,249.6635,1.778105,188.5254,1.736584,1,2,1
7499,1451304299000000000,2.592000,509.7363,2,2,1,263.4261,8.368238,329.5309,2.444879,...,0.306889,2,1,249.6635,1.778105,188.5254,1.736584,1,2,1


In [3]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

X_tsa, index = time_segments_aggregate(df, interval=1000000000, time_column='timestamp')

In [4]:
imp = SimpleImputer()
X_imp = imp.fit_transform(X_tsa)

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1)) ## for the gradients to converge faster == try 0-1 StandardScaler()
X_scl = scaler.fit_transform(X_imp)
X = X_scl
X_scl.shape

(7500, 51)

In [6]:
y = np.array(y, dtype=str)
y_binary = []
for i in y:
    if i == "Normal":
        y_binary.append(False)  #false == normal
    else:
        y_binary.append(True)   #true == attack
        
y_binary = np.array(y_binary)
y_binary

array([False, False, False, ..., False, False, False])

In [7]:
#select = SelectKBest(chi2, k=20)
#X_chi2 = select.fit_transform(X_scl, y)
#X_chi2[0]

In [8]:
feature_names = np.array(df.columns, dtype=str)
feature_names

array(['timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102',
       ' AIT201', 'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201',
       ' P202', 'P203', ' P204', 'P205', 'P206', 'DPIT301', 'FIT301',
       'LIT301', 'MV301', 'MV302', ' MV303', 'MV304', 'P301', 'P302',
       'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403',
       'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501',
       'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502',
       'PIT503', 'FIT601', 'P601', 'P602', 'P603'], dtype='<U9')

In [9]:
#filter = select.get_support()
#len(filter)

In [10]:
#chi2_features = []
#for i in range(len(feature_names)):
#    if filter[i]:
#        chi2_features.append(feature_names[i])
#print(chi2_features)
#print(len(chi2_features))

In [11]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(df, y_binary, len(feature_names))
print(str(len(cor_feature)), 'selected features')
print(str(len(cor_support)), 'cor_support')

52 selected features
52 cor_support


/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(df)
chi_selector = SelectKBest(chi2, k=len(feature_names))
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = df.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
print(str(len(chi_support)), 'chi_support')

52 selected features
52 chi_support


/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [13]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=len(feature_names), step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = df.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(str(len(rfe_support)), 'rfe_support')

/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


52 selected features
52 rfe_support


In [14]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), max_features=len(feature_names))
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = df.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')
print(str(len(embeded_lr_support)), 'embeded_lr_support')

/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


29 selected features
52 embeded_lr_support


In [15]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=len(feature_names))
embeded_rf_selector.fit(df, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = df.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')
print(str(len(embeded_rf_support)), 'embeded_rf_support')

13 selected features
52 embeded_rf_support


In [17]:
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(len(feature_names))

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,timestamp,True,True,True,True,True,5
2,P102,True,True,True,True,True,5
3,LIT401,True,True,True,True,True,5
4,LIT301,True,True,True,True,True,5
5,LIT101,True,True,True,True,True,5
6,FIT201,True,True,True,True,True,5
7,AIT503,True,True,True,True,True,5
8,AIT502,True,True,True,True,True,5
9,AIT501,True,True,True,True,True,5
10,AIT402,True,True,True,True,True,5
